In [1]:
import requests
# pretty print is used to print the output in the console in an easy to read format
from pprint import pprint
from datetime import datetime
from web3 import Web3
import pandas as pd
import matplotlib.pyplot as plt
import time
import re
import numpy as np

In [2]:
# transaction ids has to be checked in etherscan
node_url = "https://alien-sparkling-gadget.discover.quiknode.pro/ea1f5f2f5471b3a0525fa85dbfcfa27a5f90e25d/"
web3 = Web3(Web3.HTTPProvider(node_url))
tx = web3.eth.get_transaction('0xffe16a6ba847b9d189b6bb28686d1418bc1f4ef150800e4c48b5c02fe3555797')
block = tx.blockNumber
ts = web3.eth.get_block(block).timestamp
datetime.utcfromtimestamp(ts)

datetime.datetime(2023, 5, 21, 14, 41, 23)

In [3]:
from web3 import Web3
from web3 import AsyncHTTPProvider
from web3.eth import AsyncEth
import asyncio
import json
import math
import nest_asyncio
nest_asyncio.apply()


In [11]:
nodes = ['https://mainnet.infura.io/v3/174e045511c742af9cbe23f6bd053402',
         'https://mainnet.infura.io/v3/0731bcfffdb844daa0972c90e98da4e6',
         'https://mainnet.infura.io/v3/6d0c853857a249878418776d98bf732a']

In [12]:
# Define providers
w3 = Web3(Web3.HTTPProvider(nodes[0]))
providers = []
providersAsync = []
for node in nodes:
    providers.append(Web3.HTTPProvider(node))
    providersAsync.append(Web3(AsyncHTTPProvider(node), modules={"eth": (AsyncEth)}))

In [13]:
# Read factory contract addresses
# Uniswap V2 factory contract address
# contract_address = '0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f'
# SushiSwap factory contract address
# contract_address = '0xC0AEe478e3658e2610c5F7A4A2E1777cE9e4f2Ac'
with open("/home/ildefons/memeprj/Articles-mev-bot-series/Part 3/FactoriesV2.json", "r") as f:
    factories = json.load(f)

In [14]:
# Define the contract ABI
factory_abi = [
    {
        "anonymous": False,
        "inputs": [
            {
                "indexed": True,
                "internalType": "address",
                "name": "token0",
                "type": "address",
            },
            {
                "indexed": True,
                "internalType": "address",
                "name": "token1",
                "type": "address",
            },
            {
                "indexed": False,
                "internalType": "address",
                "name": "pair",
                "type": "address",
            },
            {
                "indexed": False,
                "internalType": "uint256",
                "name": "",
                "type": "uint256",
            },
        ],
        "name": "PairCreated",
        "type": "event",
    }
]

In [16]:
def getPairEvents(contract, fromBlock, toBlock):
    toBlockPrime = toBlock
    fetchCount = 0

    # Then, recursively fetch events in smaller time intervals
    def getEventsRecursive(contract, _from, _to):
        try:
            events = (
                contract.events.PairCreated()
                .create_filter(fromBlock=_from, toBlock=_to)
                .get_all_entries()
            )
            print("Found ", len(events), " events between blocks ", _from, " and ", _to)
            nonlocal fetchCount
            fetchCount += len(events)
            return events
        except ValueError:
            print("Too many events found between blocks ", _from, " and ", _to)
            midBlock = (_from + _to) // 2
            return getEventsRecursive(contract, _from, midBlock) + getEventsRecursive(
                contract, midBlock + 1, _to
            )

    return getEventsRecursive(contract, fromBlock, toBlockPrime)

In [17]:
# Fetch list of pools for each factory contract
pairDataList = []
for factoryName, factoryData in factories.items():
    events = getPairEvents(
        w3.eth.contract(address=factoryData["factory"], abi=factory_abi),
        0,
        w3.eth.block_number,
    )
    print(f"Found {len(events)} pools for {factoryName}")
    for e in events:
        pairDataList.append(
            {
                "token0": e["args"]["token0"],
                "token1": e["args"]["token1"],
                "pair": e["args"]["pair"],
                "factory": factoryName,
            }
        )

Too many events found between blocks  0  and  17735456
Found  0  events between blocks  0  and  8867728
Too many events found between blocks  8867729  and  17735456
Too many events found between blocks  8867729  and  13301592
Too many events found between blocks  8867729  and  11084660
Found  0  events between blocks  8867729  and  9976194
Too many events found between blocks  9976195  and  11084660
Found  2887  events between blocks  9976195  and  10530427
Too many events found between blocks  10530428  and  11084660
Found  4850  events between blocks  10530428  and  10807544
Found  8489  events between blocks  10807545  and  11084660
Too many events found between blocks  11084661  and  13301592
Too many events found between blocks  11084661  and  12193126
Too many events found between blocks  11084661  and  11638893
Found  8328  events between blocks  11084661  and  11361777
Found  3655  events between blocks  11361778  and  11638893
Found  4098  events between blocks  11638894  and 